# **Using M3d-CAM for classification**

In this demo you will learn how to use M3d-CAM for classification using a resnet152. We will use a subset of the famous [Cats vs Dogs Dataset](https://github.com/Karol-G/gcam_cat_dog_examples) for this demo. \\

This demonstration was made using Google Colab and probably won't work if you are not using Colab.

# Preparation

Clone the Cats vs Dogs repository and set up the data structure:

In [2]:
!git clone https://github.com/Karol-G/Gcam_cat_dog_examples.git

Cloning into 'Gcam_cat_dog_examples'...
remote: Enumerating objects: 33, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 33 (delta 3), reused 27 (delta 0), pack-reused 0
Unpacking objects: 100% (33/33), done.


Install M3d-CAM:

In [6]:
pip install medcam

     |████████████████████████████████| 81kB 8.1MB/s 
     |████████████████████████████████| 47.4MB 94kB/s 


# Model & dataloader setup

Next we set up our resnet152 model and the dataloader for loading the cat and dog images:

In [15]:
from torchvision import models, transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torch
import cv2

# Setup the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = models.inception_v3(pretrained=True)
model.to(device=device)
model.eval()

def load_image(image_path):
    raw_image = cv2.imread(image_path)
    raw_image = cv2.resize(raw_image, (256,) * 2)
    image = transforms.Compose(
        [
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ]
    )(raw_image[..., ::-1].copy())
    image = image.to(device)
    return image

# Load the dataset
dataset = ImageFolder('/content/Gcam_cat_dog_examples/dataset', loader=load_image)
# Set up the dataloader
data_loader = DataLoader(dataset, batch_size=1, shuffle=False)

# Injecting M3d-CAM into resnet152

The beauty of M3d-CAM is that you only need to insert a single line of code (or two if you count the import) for everything to work:

In [16]:
from medcam import medcam

model = medcam.inject(model, output_dir='attention_maps', backend='gcam', layer='layer11', label= 'best', save_maps=True)

After your model is injected with M3d-CAM it will still behave as it would normally do. So even if you have a big and complex project nothing will break and it will run as it always did. \\
The only difference is that every time the `model.forward()` of your model is called attention maps will be generated for your current input and automatically saved to `output_dir`. \\
The output of your model stays the same as before the injection. (Of course you can change this behavior and return the attention maps instead by setting `replace=True` during the injection).

Now to generate some attention maps we will call the `model.forward()` with the cat and dog images a couple times:

In [17]:
for batch in data_loader:
    output = model(batch[0])

KeyError: ignored

Now you can display the generated attention maps in colab with:

In [ ]:
from IPython.display import Image

Image('/content/attention_maps/layer4/01E392EE-69F9-4E33-BFCE-E5C968654078.jpeg')

In [8]:
medcam.get_layers(model)

['conv1',
 'bn1',
 'relu',
 'maxpool',
 'layer1.0.conv1',
 'layer1.0.bn1',
 'layer1.0.conv2',
 'layer1.0.bn2',
 'layer1.0.conv3',
 'layer1.0.bn3',
 'layer1.0.relu',
 'layer1.0.downsample.0',
 'layer1.0.downsample.1',
 'layer1.0.downsample',
 'layer1.0',
 'layer1.1.conv1',
 'layer1.1.bn1',
 'layer1.1.conv2',
 'layer1.1.bn2',
 'layer1.1.conv3',
 'layer1.1.bn3',
 'layer1.1.relu',
 'layer1.1',
 'layer1.2.conv1',
 'layer1.2.bn1',
 'layer1.2.conv2',
 'layer1.2.bn2',
 'layer1.2.conv3',
 'layer1.2.bn3',
 'layer1.2.relu',
 'layer1.2',
 'layer1',
 'layer2.0.conv1',
 'layer2.0.bn1',
 'layer2.0.conv2',
 'layer2.0.bn2',
 'layer2.0.conv3',
 'layer2.0.bn3',
 'layer2.0.relu',
 'layer2.0.downsample.0',
 'layer2.0.downsample.1',
 'layer2.0.downsample',
 'layer2.0',
 'layer2.1.conv1',
 'layer2.1.bn1',
 'layer2.1.conv2',
 'layer2.1.bn2',
 'layer2.1.conv3',
 'layer2.1.bn3',
 'layer2.1.relu',
 'layer2.1',
 'layer2.2.conv1',
 'layer2.2.bn1',
 'layer2.2.conv2',
 'layer2.2.bn2',
 'layer2.2.conv3',
 'layer2.2.b

# Some further notes

As M3d-CAM offers multiple methods of visualization (*backends*) you can simply change the backend keyword to one of the following: \\
- *gbp* (Guided Backpropagation)
- *gcam* (Grad-Cam, default)
- *ggcam* (Guided Grad-Cam)
- *gcampp* (Grad-Cam++)

The layer keyword tells M3d-CAM for which layer the attention maps should be generated. You can also set the layer to 'auto' (the default setting) and M3d-CAM will choose the last convolutional layer which is what you want in most cases. However this is still experimental and won't always choose the correct layer. But it works in most cases. \\
You can print all layers of a model with `medcam.get_layers(model)` if you don't know the layer names. However you cannot generate attention maps from every layer. \\

Furthermore the label keyword tells M3d-CAM on which class label it should focus. The default setting is 'best' which always selects the class with the highest probability. Alternativly you can set the label manually as a number.